[1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
# ==================================================
# [1-1] 모듈 로딩
# ==================================================
# 기본 모듈
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib

# ML 데이터셋 및 학습 관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# ML 모델 성능 지표 관련 모듈
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import root_mean_squared_error, r2_score

# CoW 활성화 : 복사와 공유 자동 설정
pd.options.mode.copy_on_write = True

# 판다스/넘파이에 지수표기법(과학적표기법) 정지 설정
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(precision=6, suppress=True)

In [4]:
# ==================================================
# [1-2] 데이터 준비 및 확인
# ==================================================
# 데이터 준비
DATA_FILE = '../Data/world_energy_co2_train_1965_2000.csv'
DATA_FILE2 = '../Data/world_energy_co2_test_2001_2013.csv'

trainDF = pd.read_csv(DATA_FILE)
testDF  = pd.read_csv(DATA_FILE2)

display(trainDF.head(3))
trainDF.info()

,year,co2,co2_per_capita,energy_per_capita,energy_per_gdp,primary_energy_consumption,coal_co2,oil_co2,gas_co2
0,1965,197.33488,4.34472,18441.73145,1.81125,785.75645,91.10069,76.35566,22.88378
1,1966,201.80255,5.16399,21466.07260,1.94501,808.99150,89.67024,80.46080,24.21385
2,1967,206.55301,5.22621,22122.39947,1.96620,836.23505,87.36961,85.60066,25.73094


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        36 non-null     int64  
 1   co2                         36 non-null     float64
 2   co2_per_capita              36 non-null     float64
 3   energy_per_capita           36 non-null     float64
 4   energy_per_gdp              36 non-null     float64
 5   primary_energy_consumption  36 non-null     float64
 6   coal_co2                    36 non-null     float64
 7   oil_co2                     36 non-null     float64
 8   gas_co2                     36 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 2.7 KB


In [ ]:
trainDF['RiskScore'] = 0.4 * trainDF['co2_per_capita'] + 0.3 * trainDF['co2'] + 0.3 * trainDF['energy_per_gdp']

KeyError: 'fossil_energy_ratio'

In [ ]:
Climate Risk Score =
0.4 × trainDF['co2_per_capita']
+ 0.3 × trainDF['fossil_energy_ratio']
+ 0.3 × trainDF['energy_per_gdp']
